In [1]:
import cv2
import numpy as np
import dlib
import matplotlib.pyplot as plt
import os
import tensorflow as tf 

In [2]:
def detect_face_landmarks(frame, face_detector, landmark_predictor):
    
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_detector(gray_frame)
    
    landmarks_list = []
    for face in faces:
        landmarks = landmark_predictor(gray_frame, face)
        landmarks_points = [(landmarks.part(n).x, landmarks.part(n).y) for n in range(68)]
        landmarks_list.append(landmarks_points)
    
    return landmarks_list

In [3]:
def extract_eyes_from_landmarks(frame, landmarks):
    
    if len(landmarks) != 68:
        raise ValueError("Facial landmarks should contain 68 points.")

    # Define the indices for the left and right eyes based on facial landmarks.
    left_eye_indices = [i for i in range(36, 42)]
    right_eye_indices = [i for i in range(42, 48)]

    # Extract left and right eye regions from the frame.
    left_eye_coords = np.array([landmarks[i] for i in left_eye_indices], dtype=np.int32)
    right_eye_coords = np.array([landmarks[i] for i in right_eye_indices], dtype=np.int32)

    # Create a mask for each eye region.
    mask_left = np.zeros_like(frame)
    mask_right = np.zeros_like(frame)
    cv2.fillPoly(mask_left, [left_eye_coords], (255, 255, 255))
    cv2.fillPoly(mask_right, [right_eye_coords], (255, 255, 255))

    # Apply the mask to the frame to get the eye regions.
    left_eye_region = cv2.bitwise_and(frame, mask_left)
    right_eye_region = cv2.bitwise_and(frame, mask_right)

    # Convert the eye regions to grayscale for further processing if needed.
    left_eye_gray = cv2.cvtColor(left_eye_region, cv2.COLOR_BGR2GRAY)
    right_eye_gray = cv2.cvtColor(right_eye_region, cv2.COLOR_BGR2GRAY)

    return [left_eye_gray, right_eye_gray]

In [4]:
def preprocess_single_image(image_path, img_size):
    # Read the image in grayscale
    img_array = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Resize the image while maintaining the aspect ratio
    desired_size = (img_size, img_size)
    height, width = img_array.shape
    aspect_ratio = width / height

    if aspect_ratio >= 1:
        new_width = desired_size[0]
        new_height = int(new_width / aspect_ratio)
    else:
        new_height = desired_size[1]
        new_width = int(new_height * aspect_ratio)

    resized_image = cv2.resize(img_array, (new_width, new_height))

    # Pad the resized image to make it square (img_size x img_size)
    pad_width = (desired_size[1] - new_height) // 2
    pad_height = (desired_size[0] - new_width) // 2
    padded_image = np.pad(resized_image, ((pad_width, pad_width), (pad_height, pad_height)), mode='constant', constant_values=0)

    # Convert the grayscale image to RGB
    rgb_image = cv2.cvtColor(padded_image, cv2.COLOR_GRAY2RGB)

    return rgb_image


In [6]:
# Load the dlib face detector and facial landmark predictor models
face_detector = dlib.get_frontal_face_detector()
landmark_predictor = dlib.shape_predictor('..\\dlib_shape_predictor\\shape_predictor_68_face_landmarks.dat')
# Load your drowsiness detection model
model = tf.keras.models.load_model('my_model.keras')

# Open the webcam
cap = cv2.VideoCapture(0)

# Create a directory to save the extracted eye images
if not os.path.exists("eye_images"):
    os.makedirs("eye_images")

frame_buffer = []
closed_eye_frames_threshold = 5

while True:
    ret, frame = cap.read()

    if not ret:
        break

    # Detect facial landmarks
    landmarks = detect_face_landmarks(frame, face_detector, landmark_predictor)

    # If a face is detected, extract eye regions from the frame
    if len(landmarks) > 0:
        eyes = extract_eyes_from_landmarks(frame, landmarks[0])  # Considering the first detected face
        left_eye, right_eye = eyes[0], eyes[1]

        # Save the extracted eye images
        cv2.imwrite(f"eye_images/left_eye_{len(frame_buffer)}.jpg", left_eye)
        cv2.imwrite(f"eye_images/right_eye_{len(frame_buffer)}.jpg", right_eye)

        left_eye = preprocess_single_image(f"eye_images/left_eye_{len(frame_buffer)}.jpg", img_size=100)
        right_eye = preprocess_single_image(f"eye_images/right_eye_{len(frame_buffer)}.jpg", img_size=100)    
        # Process the eye images with your drowsiness detection model
        resized_image_l = cv2.resize(left_eye, (100, 100))
        resized_image_r = cv2.resize(right_eye, (100, 100))
        # Reshape the preprocessed image to match the input shape of the model
        preprocessed_image_l = resized_image_l.reshape(-1, 100, 100, 3)
        preprocessed_image_r = resized_image_r.reshape(-1, 100, 100, 3)

        prediction_left = model.predict(preprocessed_image_l)  # Modify this line as per your model's input requirements
        prediction_right = model.predict(preprocessed_image_r)
        print(prediction_left,'right==',prediction_left)
        if (prediction_left < 0.5 and prediction_right < 0.5):  # Assuming 0.5 as the threshold for closed eyes
            frame_buffer.append(True)
        else:
            frame_buffer.append(False)

        if len(frame_buffer) > closed_eye_frames_threshold:
            if all(frame_buffer[-closed_eye_frames_threshold:]):
                print("ALERT: Eyes closed for several frames!")
                # Trigger your alert mechanism here, e.g., sending an email, playing a sound, etc.
                frame_buffer.clear()

    # Exit the loop when 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('e'):
        break

# Release the webcam and close all windows
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 58ms/step
[[0.7970168]] right== [[0.7970168]]
1/1 [==============================] - 0s 49ms/step
[[0.7402576]] right== [[0.7402576]]
1/1 [==============================] - 0s 50ms/step
[[0.78582263]] right== [[0.78582263]]
1/1 [==============================] - 0s 57ms/step
[[0.7693068]] right== [[0.7693068]]
1/1 [==============================] - 0s 49ms/step
[[0.7704585]] right== [[0.7704585]]
1/1 [==============================] - 0s 58ms/step
[[0.72957504]] right== [[0.72957504]]
1/1 [==============================] - 0s 49ms/step
[[0.7200378]] right== [[0.7200378]]
1/1 [==============================] - 0s 57ms/step
[[0.6978913]] right== [[0.6978913]]
1/1 [==============================] - 0s 49ms/step
[[0.7620395]] right== [[0.7620395]]
1/1 [==============================] - 0s 58ms/step
[[0.70578855]] right== [[0.70578855]]
1/1 [==============================] - 0s 66ms/step
[[0.7163157]] right== [[0.7163157]]
1/1 [=====================

KeyboardInterrupt: 